# Clustering algorithms

In this notebook, we will learn about two kinds of clustering algorithms (k-means and agglomerative clustering), but also about the intuition behind how clustering works. We will also perform clustering on text documents.

**Table of Contents**  

1. [Preparing Synthetic Data](#sec1)
2. [K-Means](#sec2)
3. [Agglomerative Clustering](#sec3)
4. [Distances used in clustering](#sec4)
5. [Clustering text documents with k-means](#sec5) 

<a id="sec1"></a>

## 1. Preparing synthetic data

We will generate a synthetic data set that has natural clusters and then will use clustering functions 
to see  how well they work. We are doing this, because otherwise it would be difficult to know whether the agorithms were able to find meaningful clusters.

In [ ]:
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn
plt.style.use('ggplot')

Here is the function that will create the synthetic data:

In [ ]:
from sklearn.datasets import make_blobs 

Let's look up the documentation for `make_blobs` to find out how to use it:

In [ ]:
help(make_blobs)

Now that we know how to call `make_blob`, we can generate a dataset. Notice the importance of the feature `random_state`. That ensures that we can reproduce the values whenever we run the notebook.

I want to create data that is groued in four clusters.

In [ ]:
X, y_true = make_blobs(n_samples=120, 
                       centers=4,
                       cluster_std=0.40, 
                       random_state=0)

print(X.shape)
print(y_true.shape)

We just created the matrix `X` that has 120 rows (one for each sample) with two columns each (meaning two features), and the vector `y_true`, that has 120 values, indicating which cluster (or blob) a data sample (or instance) belongs to.

In [ ]:
y_true

Let's check that we have the same number of values in each blob, by counting unique values with np:

In [ ]:
import numpy as np

np.unique(y_true, return_counts=True)

X is a two-dimensional array, every point has two feature values (so that we can plot it on a 2D plane):

In [ ]:
X[:10]

We can plot the data samples in X to see the clusters.

In [ ]:
plt.figure(figsize=(9,6))
plt.scatter(X[:, 0], X[:, 1], s=50); # s is the size of the dots in the plot

It is pretty clear that the data points belong to four different clusters. Let's use the various clustering algorithms we have learned about to find out how the data will be clustered.

<a id="sec2"></a>
## 2. K-Means

K-means clustering will divide data in k clusters, each with its own center. The algorithm works iteratively  
to group together data points that are spatially closer to one-another.

We will use the Kmeans algorithm that is implemented within the `sklearn` package.

All `sklearn` algorithms have the same API involving these steps:

1. initialize the model
2. fit the data to the model
3. validate the model or predict new values

Each of these steps is usually one single line of code.

In [ ]:
from sklearn.cluster import KMeans 

kmeans = KMeans(n_clusters=4)  # step 1: initialize the model
kmeans.fit(X)                  # step 2: fit the data to the model
y_kmeans = kmeans.predict(X)   # step 3: prediction

Let's plot the results in order to see whether the clustering worked:

In [ ]:
def plot_kmeans(kmeans, y_kmeans):
    """Helper function to plot the results of the kmeans clustering"""
    # Plot the points
    plt.figure(figsize=(9,6))
    plt.scatter(X[:, 0], # first column 
                X[:, 1], # second column
                c=y_kmeans, # indices for the color mapping
                s=50,  # size of a dot
                cmap='plasma') # color palette
    
    # Plot the cluster centers
    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
    plt.show()

plot_kmeans(kmeans, y_kmeans)

In [ ]:
# The variable center stores the points that are the centroids of the clusters
centers

Look up the coordinates of the centers in the plot and identify the clusters (first row is cluster 0, second row is cluster 1, and so on).

### Tighter data
We can play with the parameters of our data generation process to create different-looking clusters; for example clusters that are not well separated, by increasing the standard deviation:

In [ ]:
X, y_true = make_blobs(n_samples=300, centers=5,
                       cluster_std=0.8, # notice this is the double of std from the first example
                       random_state=0)

In [ ]:
plt.figure(figsize=(9,6))
plt.scatter(X[:, 0], X[:, 1], s=50); # s is the size of the dots in the plot

Let's see how good the k-means clustering algorithm will perform this time:

In [ ]:
# Perform clustering
kmeans = KMeans(n_clusters=5) 
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

# Do visualization
plot_kmeans(kmeans, y_kmeans)

It worked pretty well.

### Changing the number of clusters

Let's see what happens when we ask it to find 2 clusters, instead of the 5 natural clusters that are in the data:

In [ ]:
# Perform clustering
kmeans = KMeans(n_clusters=2) 
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

# Do visualization
plot_kmeans(kmeans, y_kmeans)

This also makes sense!

Let's ask about 3 clusters:

In [ ]:
# Perfom Clustering
kmeans = KMeans(n_clusters=3) 
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

# Do visualization
plot_kmeans(kmeans, y_kmeans)

The agorithm does the right thing even when we ask to find less clusters, thus, it's up to us to decide how many we want. For that, we can use the "Elbow Point"

### Visualizing the Elbow Point

The K-means algorithm tries to find centroids that minimise the **inertia**, which is the within-cluster sum-of-squares criterion. 

That is, for every point x, we find the square of the distance from its cluster's centroid and take the overall sums of all points in the cluster. The smaller this sum, the tighter the cluster, the better the clustering quality. The optimal number of clusters is where this error stops decreasin, which in a graph is shown as an **elbow point**.

In [ ]:
def elbow_viz(X, num):

    # Perform clustering from 1 to num
    cost =[] 
    for i in range(1, num): 
        KM = KMeans(n_clusters = i) 
        KM.fit(X) 
          
        # calculates squared error for the clustered points 
        cost.append(KM.inertia_)      
      
    # plot the cost against K values 
    plt.plot(range(1, num), cost, color ='g', linewidth ='3') 
    plt.xlabel("Value of K") 
    plt.ylabel("Squared Error (Cost)") 
    plt.show() 

# Call function
elbow_viz(X, 11)

We definitively see a sharp decrease at k=2 (our data has the best separation at 2 clusers). However, we can also see that the inertia continues decreasing and it stops doing so at k=5 (between 4 and 6). 5 is the natural number of clusters in this synthetic dataset.

### Explorations

Let's explore the Elbow method a bit more. 

**Create three clusters**

In [ ]:
# Create synthetic data
X, y_true = make_blobs(n_samples=150, 
                       centers=3,
                       cluster_std=0.40, 
                       random_state=0)

plt.scatter(X[:, 0], X[:, 1], s=50); 

In [ ]:
elbow_viz(X, 10)

In this occasion, the number k=3 is very clear from the Elbow visualization.

**Create 7 clusters**

In [ ]:
# Create synthetic data
X, y_true = make_blobs(n_samples=210, 
                       centers=7,
                       cluster_std=0.30, 
                       random_state=0)

plt.scatter(X[:, 0], X[:, 1], s=50); 

In [ ]:
elbow_viz(X, 14)

<a id="sec3"></a>
## 3. Agglomerative Clustering

In Kmeans clustering, we provide the number of clusters and then the algorithm partitions the data, so that they are clustered around a center point. 
In agglomerative clustering, the data points are grouped together based on distance from one another, and we can decide how many clusters we want, once we see how data are grouped together.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

# linkage is the the function that performs the clustering
Z = linkage(X, 
            method='single', 
            metric='euclidean')

In [ ]:
# create string labels for the data samples, to show in the dendrogram
ticks = [f"el_{i}_group={el}" for i, el in enumerate(y_true)]

Draw the dendrogram (takes a few seconds):

In [ ]:
def draw_dendrogram(Z, ticks):
    """Helper function to draw a dendrogram for testing purposes.
    """
    plt.figure(figsize=(25, 10))
    plt.title('Hierarchical Clustering')
    plt.xlabel('samples')
    plt.ylabel('distance')
    dendrogram(
        Z,
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
        labels = ticks
    )
    plt.show()
    
draw_dendrogram(Z, ticks)    

Something this makes clear is that visualizing the dendrogram is useful, but not that useful when we have many points. The dataset in the chart has a lot points, and it's currently hard to see the labels of the points.

### Create a smaller dataset
Let's create a smaller dataset that also is a bit more separated.

In [ ]:
X, y_true = make_blobs(n_samples=80, centers=3,
                       cluster_std=0.4, random_state=0)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=50);

Let's try the hierarchical clustering again:

In [ ]:
Z = linkage(X, method='single', metric='euclidean')

And now plot the dendrogram:

In [ ]:
ticks = ['el_{}_group={}'.format(i, el) for i, el in enumerate(y_true)]

draw_dendrogram(Z, ticks)

For this dataset, even the labels are visible. In fact, we can see how the original groups (notice the second part of name) are clustered together. The first cluster (left) contains points from group=1, the middle cluster contains points from group=0, and the cluster on the right contains points from group=2.

A final example:

In [ ]:
X, y_true = make_blobs(n_samples=60, centers=4,
                       cluster_std=0.3, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
Z = linkage(X, method='single', metric='euclidean')

ticks = ['el_{}_group={}'.format(i, el) for i, el in enumerate(y_true)]

draw_dendrogram(Z, ticks)

<a id="sec4"></a>
## 4. Distance metrics used in clustering

Clustering algorithms use various "distance metrics" to measure how far apart two points are in space. 

In `scipy` we can calculate the **pairwise distance** of every two data points with the function `pdist`:

In [ ]:
from scipy.spatial.distance import pdist

X, y_true = make_blobs(n_samples=80, centers=3,
                       cluster_std=0.4, random_state=0)

xdist = pdist(X, 'euclidean')
xdist.shape

**Question:** Why is the shape of the distance vector 3160? The dataset X has 80 points. What formula involving the data size n=80 will produce the result 3160? 

**Your answer:** 

Let's look up the first 5 values of X first:

In [ ]:
X[:5]

Now let's look up the first 5 values of xdist:

In [ ]:
xdist[:5]

The first value in the xdist is the distance between points X[0] and X[1], that distance is small, because, as you can see below the values for X[0] and X[1], they are close.

In [ ]:
X[0], X[1]

However, the second value, 3.97438162, means that X[0] and X[2] are further apart, see values below:

In [ ]:
X[0], X[2]

We can see this even better if we do a scatterplot of only these five points and label the points with their index:

In [ ]:
# create an array with five points
X5 = X[:5]

# make scatterplot
plt.scatter(X5[:, 0], X5[:, 1], s=50)

# show text next to each point (notice the delta-s, to be slightly off the point)
for i in range(5):
    plt.text(X5[i,0]-0.05, X5[i,1]-0.05, s=f"{i}")

This shows clearly that the point 0 is very close to points 1 and 4 and somewhat to point 3, but is further away from point 2. As a reminder, these four distances (of the point 0 to the other 4 points) are captured by the first elements of xdist:

In [ ]:
xdist[:4]

**More distance metrics**

We only showed here the euclidean distance, but the function `pdist`, short for pairwise distance, has implementations of more than 20 different distance metrics, they are listed [on this page](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html). 

**Distance matrix**

Once we create a vector of pairwise distances with `pdist`, we can display them as a matrix, to create the so-called distance matrix, where we can look up the distance between any two points.

In [ ]:
from scipy.spatial.distance import squareform

distM = squareform(xdist) # make a matrix
distM.shape

A distance matrix can be visualiazed through a heatmap. We can easily build one with seaborn:

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,10))
sns.heatmap(distM, cmap="Purples")
plt.show()

<a id="sec6"></a>

<a id="sec5"></a>
## 5. Clustering documents with k-means

We'll try clustering with a famous text dataset called 20newsgroups. Read a bit about its content on [this page](http://qwone.com/~jason/20Newsgroups/).

Because this is a famous dataset, `sklearn` already knows how to read its content from files. If you are curious about the files, download the ZIP archive from the website. Careful, it's a big file.

First, sklearn will get the dataset:

In [ ]:
import sklearn.datasets
all_data = sklearn.datasets.fetch_20newsgroups(subset='all')
print(len(all_data.filenames))

It contains 18846 files in total, so, it's a big dataset.  

Documents are grouped in classes (a class is known as a target). Let's see these classes:

In [ ]:
print(all_data.target_names)

Because this dataset is usually used for text classification, it is divided in two parts: "train" data and "test" data. Both of these groups have examples with labels (the category where a piece of news belongs), but the algorithm will learn its model from the train data and then it is tested for accuracy (how well is doing) on the test data.

In [ ]:
train_data = sklearn.datasets.fetch_20newsgroups(subset='train')
print(len(train_data.filenames))

In [ ]:
test_data = sklearn.datasets.fetch_20newsgroups(subset='test')
print(len(test_data.filenames))

In this dataset, 60% of files are assigned to training and 40% are assigned to testing.

One doesn't need to work with all 20 groups at once, we can choose to focus on a subset:

In [ ]:
groups = ['alt.atheism', 'comp.graphics', 'rec.autos', 
          'rec.sport.baseball', 'sci.med', 'talk.politics.guns']


train_data = sklearn.datasets.fetch_20newsgroups(subset='train', categories=groups)
test_data = sklearn.datasets.fetch_20newsgroups(subset='test', categories=groups)
print(len(train_data.filenames), len(test_data.filenames))

### Representing documents as vectors

Clustering works with vectors of numbers, thus, all documents need to be converted into such vectors. This is what we had to do in Assignment 6 as well. This process is known as vectorizing, and we used some vectorizers in Assignment 6. However, our features there were simple. When we are working with free text, the process needs to be more involved. Below we show a vectorizer that does two things together:

1. stemming of words (reducing a word to its stem, for example: "working"--> "work", etc.), 
2. calculation of tf*idf (the product of term frequency and the inverse document frequency) (for the moment, don't worry what these are, we'll discuss them in class briefly).

In [ ]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfVectorizer

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

**Choices**  
In addition to removing stop-words, getting stems, ignoring unicode characters that cannot be decoded, we are also dropping all words that appear less than 10 times (this will remove spelling mistakes or unusual acronyms), as well as words that appear in 50% of the documents. All these efforts are for the goal of reducing the dimensions of the problem (by having a smaller number of features).

In [ ]:
vectorizer = StemmedTfidfVectorizer(min_df=10, max_df=0.5, 
                                    stop_words='english', 
                                    decode_error='ignore')

Let's fit the data to this vectorizer, it might take a few seconds to complete.

In [ ]:
vectorized = vectorizer.fit_transform(train_data.data)
vectorized

**Explanation:**

- there are 3395 rows in the matrix, each of them representing a document (an email)
- there are 5068 columns in the matrix, each of them representing a feature (a stemmed word)
- such a matrix would have 17,205,860 cells, if not in sparse format. However, it's clear that each document will have only a few dozen words, thus most of the values in each row are 0. In fact, there is a toal of 272,628 non-zero values in this matrix.
- each non-zero cell contain the tf*idf value of the feature in a document.

We can use attributes and methods of the vectorizer to access more information about the model we have built. For example, we can look at the words (or the vocabulary of the corpus):

In [ ]:
len(vectorizer.vocabulary_)

This variable is a dictionary, so we can loop up some of its keys:

In [ ]:
list(vectorizer.vocabulary_.items())[:10]

The words in the vocabulary are the feature names, we can verify that:

In [ ]:
vectorizer.get_feature_names_out()[2730:2745]

### Apply Clustering

Lets initialize the clustering algorithm:

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 12
km = KMeans(n_clusters=num_clusters, 
            init='random', 
            n_init=3, 
            verbose=1, 
            random_state=3)

We can fit the model to the data, meaning, we'll find the parameters for the 12 clusters (coordinates of the centroids). I set **verbose=1**, so that we can see the process of iteration in order to converge to cluster centers.

In [ ]:
km.fit(vectorized)

The variable `labels_` will indicate for every document the number of cluster to which the document has been assigned.

In [ ]:
km.labels_

This array tells us that the first document was assigned to cluster 26, the second document to cluster 12, the third document to cluster 25, etc.

We can check that there is a label assigned to every document, by printing the lengths of the list:

In [ ]:
len(km.labels_)

We can look at all centers, there are large vectors:

In [ ]:
km.cluster_centers_

In [ ]:
km.cluster_centers_.shape

That is, we have 12 centers, each a vector of 5068 dimensions.

Let's print out one centroid, it should be a long array of numbers (tf-idf values) for different features(words) that are relevant to the cluster. Values 0 mean that the feature is not present. 

In [ ]:
print(list(km.cluster_centers_[0])[:50])

### Explore Clusters

Let's see how many documents are in each cluster:

In [ ]:
from collections import Counter
labelsCnt = Counter(km.labels_)

for k in sorted(labelsCnt):
    print(k, labelsCnt[k])

The data seem to be spread out across all clusters, but some clusters are much bigger than others, for example, cluster 3.

Each cluster stores the indices of the documents assigned to a cluster. Let's get the indices for one cluster:

In [ ]:
# we will find the document indices that were assigned the label 1

indices = [i for i in range(len(km.labels_)) if km.labels_[i] == 1]
len(indices)

In [ ]:
print(indices)

These indices allow us to access the documents and their labels, for example, here is how to get the text of the document:

In [ ]:
train_data.data[indices[0]]

And here is how to get the that target label and its name

In [ ]:
train_data.target[indices[0]], train_data.target_names[train_data.target[indices[0]]]

Using this relationship between target and target_names, it is possible for us to look at the original labels of all documents in Cluster 1:

In [ ]:
targetNames = [train_data.target_names[train_data.target[docID]] for docID in indices]
Counter(targetNames).most_common()

So, in this cluster, there are 50 emails from 'rec.autos', 22 emails from 'comp.graphics' and 9 emails from 'sci.med'. We will do this for all the clusters, let's checks where there instances are coming from:

In [ ]:
for label in labelsCnt:
    indices = [i for i in range(len(km.labels_)) if km.labels_[i] == label]
    targetNames = [train_data.target_names[train_data.target[docID]] for docID in indices]
    print(f"Cluster: {label}")
    print(Counter(targetNames).most_common())

As we can see, some of these clusters are really good, because they contain emails from a single newgroup (such as baseball) or the majority of documents from one newsgroup. Meanwhile, a few other clusters have documents from several newsgroups. This indicates that we might need to increase the number of clusters to get clusters that are more homogenous.

**Conclusion:** This example shows that even when working with messy text documents (such as emails), it's possible to perform clustering and find meaningful clusters. Remember, when we did the clustering, we did not provide any labels to the model, this is unsupervised learning. Thus, the groups that we see are automatically coming from the similarity of the documents. 